In [1]:
# Import packages
import os
import random
from PIL import Image
from torch import nn
import torch 
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import json 
from src.config import PROCESSED_DATA_DIR, RAW_DATA_DIR
import torch.optim as optim

from loguru import logger

from utils import train_validate_model, modify_model_output, test_model

import time
import wandb

image_path = PROCESSED_DATA_DIR / "pizza_hamburger_hotdog_20_percent"
train_dir = image_path / 'train' 
test_dir = image_path / 'test'
valid_dir = image_path / 'valid'

device ='cuda' if torch.cuda.is_available() else 'cpu'

2024-06-23 13:01:05.063 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Git\hamburger-hotdog-pizza-classifier


In [2]:
wandb.init(project='pizza_hamburger_hotdog_20_percent')

wandb: Currently logged in as: dtiourine. Use `wandb login --relogin` to force relogin


In [3]:
config_path = 'params.json'

with open(config_path, 'r') as config_file:
   config = json.load(config_file)
#print(json.dumps(config, indent=2))

In [4]:
# Load hyperparams from params.json

learning_rate = config['model_params']['learning_rate']
batch_size = config['model_params']['batch_size']
num_epochs = config['model_params']['num_epochs']
dropout_rate = config['model_params']['dropout_rate']
optimizer = config['model_params']['optimizer']
loss_function = config['model_params']['loss_function']
metrics = config['model_params']['metrics']
output_shape = config['model_params']['output_shape']

In [5]:
# Prepare data into dataloader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
])

train_data = ImageFolder(train_dir, transform=transform)
valid_data = ImageFolder(valid_dir, transform=transform)
test_data = ImageFolder(test_dir, transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Start with Pre-Trained Models

Let's choose some popular CNN architectures as a starting point

First, we need to load these pretrained models

In [6]:
num_classes = 3
alexnet = modify_model_output('alexnet', num_classes, device)
vgg16 = modify_model_output('vgg16', num_classes, device)
resnet50 = modify_model_output('resnet50', num_classes, device)

## 1. Trying ResNet50

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.3)
model_name = "ResNet50"
date_time = time.time()
logger.add(f"logs/{model_name}/training_log-{date_time}.log", format="{time} {level} {message}", level="INFO")
train_validate_model(num_epochs=10, model=resnet50, train_loader=train_loader, valid_loader=valid_loader, criterion=criterion, optimizer=optimizer, device=device)

Training Epoch 1/10: 100%|██████████| 5/5 [00:01<00:00,  3.55it/s]


2024-06-23 13:01:10.038 | INFO     | utils:train_validate_model:88 - Epoch 1, Train Loss: 1.0873, Train Accuracy: 39.26%


Validation Epoch 1/10: 100%|██████████| 3/3 [00:00<00:00,  6.09it/s]


2024-06-23 13:01:10.531 | INFO     | utils:train_validate_model:111 - Epoch 1, Validation Loss: 1.1012, Validation Accuracy: 30.56%


Training Epoch 2/10: 100%|██████████| 5/5 [00:01<00:00,  3.33it/s]


...

2024-06-23 13:01:23.576 | INFO     | utils:train_validate_model:111 - Epoch 9, Validation Loss: 1.0385, Validation Accuracy: 61.67%


Training Epoch 10/10: 100%|██████████| 5/5 [00:01<00:00,  4.74it/s]


2024-06-23 13:01:24.632 | INFO     | utils:train_validate_model:88 - Epoch 10, Train Loss: 1.0305, Train Accuracy: 67.04%


Validation Epoch 10/10: 100%|██████████| 3/3 [00:00<00:00,  5.03it/s]

2024-06-23 13:01:25.228 | INFO     | utils:train_validate_model:111 - Epoch 10, Validation Loss: 1.0346, Validation Accuracy: 61.67%


## 2. Trying VGG16

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.3)
logger.add("training_log.log", format="{time} {level} {message}", level="INFO")
train_validate_model(num_epochs=20, model=vgg16, train_loader=train_loader, valid_loader=valid_loader, criterion=criterion, optimizer=optimizer, device=device)

Training Epoch 1/20: 100%|██████████| 5/5 [00:01<00:00,  3.66it/s]


2024-06-23 13:01:26.610 | INFO     | utils:train_validate_model:88 - Epoch 1, Train Loss: 1.1994, Train Accuracy: 29.26%


Validation Epoch 1/20: 100%|██████████| 3/3 [00:00<00:00,  5.49it/s]


2024-06-23 13:01:27.166 | INFO     | utils:train_validate_model:111 - Epoch 1, Validation Loss: 1.0142, Validation Accuracy: 56.11%


Training Epoch 2/20: 100%|██████████| 5/5 [00:01<00:00,  2.86it/s]


...

2024-06-23 13:02:07.471 | INFO     | utils:train_validate_model:111 - Epoch 19, Validation Loss: 0.2908, Validation Accuracy: 88.33%


Training Epoch 20/20: 100%|██████████| 5/5 [00:01<00:00,  3.54it/s]


2024-06-23 13:02:08.901 | INFO     | utils:train_validate_model:88 - Epoch 20, Train Loss: 0.1543, Train Accuracy: 95.56%


Validation Epoch 20/20: 100%|██████████| 3/3 [00:01<00:00,  2.69it/s]

2024-06-23 13:02:10.014 | INFO     | utils:train_validate_model:111 - Epoch 20, Validation Loss: 0.2889, Validation Accuracy: 89.44%


## 3. Trying AlexNet

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.3)
train_validate_model(num_epochs=10, model=alexnet, train_loader=train_loader, valid_loader=valid_loader, criterion=criterion, optimizer=optimizer, device=device)

AttributeError: 'int' object has no attribute '_name'

In [9]:
import torchvision.models as models
vgg11 = models.vgg11(weights='DEFAULT')
num_features = vgg11.classifier[6].in_features
vgg11.classifier[6] = nn.Linear(num_features, 3)

# Training/Validation Loop
train_validate_model(num_epochs=10, train_loader=train_loader, valid_loader=valid_loader, model=model, criterion=criterion, optimizer=optimizer, device=device)

NameError: name 'model' is not defined

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.3)

train_validate_model(num_epochs=10, train_loader=train_loader, valid_loader=valid_loader, model=model, criterion=criterion, optimizer=optimizer, device=device)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import vgg16, VGG16_Weights
from torchvision.models import alexnet, AlexNet_Weights
from torchvision.models import incep

In [24]:
#model = resnet50(weights=ResNet50_Weights.DEFAULT).to(device)
backbone = model.features
#backbone[-1].out_channels
backbone

AttributeError: 'ResNet' object has no attribute 'features'

In [28]:
num_classes = 3
alexnet = modify_model_output('alexnet', num_classes)
vgg16 = modify_model_output('vgg16', num_classes)
resnet50 = modify_model_output('resnet50', num_classes)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\MLDev/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:07<00:00, 32.5MB/s] 
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\MLDev/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:22<00:00, 25.0MB/s] 
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\MLDev/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:03<00:00, 33.5MB/s] 

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights
model = resnet50(weights=ResNet50_Weights.DEFAULT).to(device)
train_validate_model(num_epochs=100, train_loader=train_loader, valid_loader=valid_loader, model=model.softmax(3), criterion=criterion, optimizer=optimizer, device=device)
#num_features = resnet50.classifier[6].in_features
#vgg11.classifier[6] = nn.Linear(num_features, 3)

In [26]:
model = models.vgg16(weights='DEFAULT')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\MLDev/.cache\torch\hub\checkpoints\vgg16-397923af.pth
 27%|██▋       | 144M/528M [00:05<00:14, 27.7MB/s] 


KeyboardInterrupt: 